In [1]:
import pandas as pd
import torch
from transformers import pipeline
from datasets import Dataset
import pandas as pd
from tqdm.auto import tqdm

C:\Users\mogal\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
folder_path= 'E:\\Case Comp\\NEST\\Training\\'
# usecase_3.to_parquet(parquet_file_path, index=False)

# Load the Parquet file back into a DataFrame
usecase_3 = pd.read_excel(folder_path + "TESTWITHSTUDY.xlsx")

usecase_3.head()

,Unnamed: 0,NCT Number,Study Title,Study URL,Acronym,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,...,Other IDs,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,Study Status
0,0,NCT03162666,Patient Outcomes Using an Expandable Spacer,https://clinicaltrials.gov/study/NCT03162666,NaN,This study is a post-market clinical follow-up...,NO,Degenerative Disc Disease,DEVICE: ALTERA,"Change in Radiographic Analysis, Global and Se...",...,RGC16-001,2/28/17,11-06-20,11-06-20,5/22/17,NaN,8/23/21,"Rush University Medical Center, Chicago, Illin...",NaN,WITHDRAWN
1,1,NCT04312048,the Effect of Isosorbide Mononitrate in Reduci...,https://clinicaltrials.gov/study/NCT04312048,NaN,The aim of the study is to evaluate the effica...,NO,IUD Insertion Pain,DRUG: Isosorbide mononitrate|DRUG: placebo,"pain during IUD insertion, intensity of patien...",...,249,4/15/20,10/25/20,11/30/20,3/18/20,NaN,1/20/21,"Ahmed Samy, Giza, 11231, Egypt",NaN,COMPLETED
2,2,NCT03144778,Durvalumab With or Without Tremelimumab in Tre...,https://clinicaltrials.gov/study/NCT03144778,NaN,This phase I trial studies how well durvalumab...,NO,Stage II Oropharyngeal Squamous Cell Carcinoma...,BIOLOGICAL: Durvalumab|BIOLOGICAL: Tremelimumab,"Change of CD8+ tumor infiltrating lymphocytes,...",...,2016-0805|NCI-2018-01199|2016-0805,07-12-17,3/15/21,3/15/21,05-09-17,NaN,3/18/21,"M D Anderson Cancer Center, Houston, Texas, 77...",NaN,COMPLETED
3,3,NCT01592721,Radiation and Cetuximab Plus Intratumoral EGFR...,https://clinicaltrials.gov/study/NCT01592721,NaN,The incorporation of novel targeted therapies ...,YES,Squamous Cell Carcinoma|Head and Neck Cancer,BIOLOGICAL: EGFR Antisense DNA,"Toxicity Rate, This is a 2-stage clinical tria...",...,CTRC 11-47|HSC20120131H,2013-04,02-01-22,02-01-22,05-07-12,9/21/22,9/21/22,"University of Pittsburgh, Pittsburgh, Pennsylv...","Study Protocol and Statistical Analysis Plan, ...",COMPLETED
4,4,NCT04253613,Laser Biostimulation in Periodontal Treatment,https://clinicaltrials.gov/study/NCT04253613,NaN,"The use of nonsurgical periodontal treatment, ...",NO,Periodontal Inflammation|Periodontal Diseases,NaN,"IL-1β level in GCF, IL-1β is a cytokine presen...",...,OMU KAEK2012/49,1/15/13,6/15/14,6/15/14,02-05-20,NaN,07-06-22,NaN,NaN,COMPLETED


In [3]:
usecase_3 = usecase_3.head(500)

In [4]:
# ✅ Use GPU if available
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)

# ✅ Define categories
categories = [
    "Oncology", "Non-Oncology"
]

# ✅ Convert Pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(usecase_3[['Conditions']])

# ✅ Batch processing function (Uses GPU)
def batch_classify(batch, threshold=0.7):
    """Classifies a batch of conditions into predefined categories."""
    results = classifier(batch['Conditions'], candidate_labels=categories, batch_size=64)  # ✅ Optimized batch inference

    classified_labels = []
    for classification in results:
        best_label = classification['labels'][0]
        best_score = classification['scores'][0]
        classified_labels.append(best_label if best_score >= threshold else "Other Rare or Unclassified")

    return {"Conditions_Category": classified_labels}

# ✅ Enable Live Progress Bar Updates
batch_size = 512
num_batches = (len(dataset) // batch_size) + 1

classified_labels = []
progress_bar = tqdm(range(num_batches), desc="Processing Conditions", unit="batch", leave=True, mininterval=0.1, ncols=100)

for i in progress_bar:
    batch_data = dataset[i * batch_size:(i + 1) * batch_size]
    if batch_data:  # Ensure batch is not empty
        result = batch_classify(batch_data)
        classified_labels.extend(result["Conditions_Category"])
        progress_bar.set_postfix(rows_processed=len(classified_labels))  # ✅ Live row count update
        progress_bar.update(1)  # ✅ Ensures progress bar moves forward in real time

# ✅ Convert Back to Pandas DataFrame
usecase_3['Conditions_Category'] = classified_labels

# ✅ Show Completion Status
total_rows = len(usecase_3)
classified_rows = usecase_3['Conditions_Category'].notnull().sum()
completion_percentage = (classified_rows / total_rows) * 100
tqdm.write(f"✅ Classification Completed: {completion_percentage:.2f}% of rows processed.")

# ✅ Display Sample Results
tqdm.write(str(usecase_3[['Conditions', 'Conditions_Category']].head()))
# usecase_3.to_csv('category_updated.csv')

Processing Conditions: 100%|██████████████████| 1/1 [05:08<00:00, 308.89s/batch, rows_processed=500]

✅ Classification Completed: 100.00% of rows processed.
                                          Conditions Conditions_Category
0                          Degenerative Disc Disease        Non-Oncology
1                                 IUD Insertion Pain        Non-Oncology
2  Stage II Oropharyngeal Squamous Cell Carcinoma...            Oncology
3       Squamous Cell Carcinoma|Head and Neck Cancer            Oncology
4      Periodontal Inflammation|Periodontal Diseases        Non-Oncology


In [5]:
usecase_3.to_csv(folder_path +'category_updated_test.csv')

In [6]:
usecase_3.columns

Index(['Unnamed: 0', 'NCT Number', 'Study Title', 'Study URL', 'Acronym',
       'Brief Summary', 'Study Results', 'Conditions', 'Interventions',
       'Primary Outcome Measures', 'Secondary Outcome Measures',
       'Other Outcome Measures', 'Sponsor', 'Collaborators', 'Sex', 'Age',
       'Phases', 'Enrollment', 'Funder Type', 'Study Type', 'Study Design',
       'Other IDs', 'Start Date', 'Primary Completion Date', 'Completion Date',
       'First Posted', 'Results First Posted', 'Last Update Posted',
       'Locations', 'Study Documents', 'Study Status', 'Conditions_Category'],
      dtype='object')